In [4]:
!pip install selenium
!pip install tqdm
!pip install openpyxl

     -------------------------------------- 242.2/242.2 KB 4.9 MB/s eta 0:00:00


In [53]:
import time # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm # for문 돌릴 때 진행상황을 %게이지로 알려준다.
from selenium import webdriver
import openpyxl

import urllib.request
import re

# 에러
from selenium.common.exceptions import NoSuchElementException # element 오류처리

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [54]:
# 액셀에서 검색할 데이터들 불러오기
load_datas = openpyxl.load_workbook("data.xlsx")
datas = load_datas.active
column_D_datas = datas['D']

crawling_names = []
for data in column_D_datas:
    crawling_names.append(data.value)

In [55]:
# v1
# ex) 신라면 블랙, 신라면을 고려안한 크롤링

# 네이버 api 사용할 때 필요한 값
client_id = "84DIX0pyMLLkcG2uuHia"
client_secret = "CRf5UovTxF"

for crawling_name in crawling_names:
    query_txt = '"' + crawling_name + '"' # 검색할 값을 따옴표 안에 넣어서 검색해주세요 (ex: "신라면 블랙") ( 따옴표 안에 넣고 검색시 이 값이 무조건 존재한 블로그들만 나타내는 걸로 알고 있어요!?)
    encText = urllib.parse.quote(query_txt) # 네이버 검색창 검색 키워드 

    blog_links = []

    for start in range(1, 200, 100): # 블로그 200개 기준으로 작성했습니다
        stringStart = str(start) # string형태여야 합니다
        url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&start="+stringStart+"&display=100" # 네이버 검색 api, start: 시작, display: 보여줄 갯수(100이 최대)

        # start(블로그 몇 번째인지?, max값 1000)와 display(한번 검색에 몇 개의 결과를 띄울건지, max값 100)조절해주기
        # start는 1, 101, 201 이렇게 시작해주세요
        # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            #print(response_body.decode('utf-8'))
        else:
            print("Error Code:" + rescode)

        body = response_body.decode('utf-8')
        body = body.replace('"','')

        #블로그 링크들 추출
        links = re.findall('link: (.*?),\ndescription',body)
        print('추출 링크 수: ',len(links),'개')

        for link in links:
            link_a = link.replace('\\','') # 링크 안의 '\\' 제거
            link_b = link_a.replace('?Redirect=Log&logNo=','/') # 링크안에 ?Redirect=Log&logNo= 제거
            blog_links.append(link_b) # 현재 링크를 배열에 추가 밑에 크롤링에서 사용

    print('총 링크 갯수:',len(blog_links),'개')

    #본문 크롤링
    path = "chromedriver.exe" # 크롬 버전에 맞는 크롬 드라이버를 실행하는 파일 위치랑 똑같은 곳에 놔둬주는게 편해요
    driver = webdriver.Chrome(path)

    # 블로그 링크를 하나씩 불러와서 크롤링
    contents = ''
    for i in tqdm(blog_links, desc="tqdm description"): 
        driver.get(i) #불러온 링크에 접속
        driver.implicitly_wait(2) # 엔진 자체에서 로딩(파싱?)되는 시간을 기다려주는 것...?
        # time.sleep(1) # driver.implicitly_wait만 써도 괜찮은지 모르겠어서 time.sleep도 넣어놓은 거예요..

        print(i+' 링크하고 있습니다')
        try: # 네이버 블로그
            driver.switch_to.frame("mainFrame")  # 1)iframe접근 
            blog_contents = driver.find_element_by_css_selector('div.se-main-container').text
            contents += blog_contents + '\n==========================================================================================\n'
        except NoSuchElementException:
            blog_contents = driver.find_element_by_css_selector('div#content-area').text
            contents += blog_contents + '\n==========================================================================================\n'
        except NoSuchElementException: 
            blog_contents = driver.find_element_by_css_selector('div.contents_style').text # 티스토리인 걸로 알고 있어요 (제가 한건데 기억이 안나요 ㅈㅅㅈㅅ;;;)
            contents += blog_contents + '\n==========================================================================================\n'
        except: # 그 외
            pass

    # 링크 내용을 txt 파일로 저장 
    text_content = open(query_txt+".txt", 'w', encoding='utf-8') # 파일이름 + 확장자명 : txt
    text_content.write(contents)  
    text_content.close()

"쌀국수 멸치맛"
추출 링크 수:  100 개
추출 링크 수:  100 개
총 링크 갯수: 200 개


tqdm description:   0%|                                                                        | 0/200 [00:00<?, ?it/s]

https://blog.naver.com/littlehaker/222591471146 링크하고 있습니다


tqdm description:   0%|▎                                                               | 1/200 [00:01<05:27,  1.65s/it]

https://blog.naver.com/dailymisa/222655671757 링크하고 있습니다


...

https://blog.naver.com/yongsan-in/222626723626 링크하고 있습니다


tqdm description:   6%|███▍                                                           | 11/200 [00:14<03:28,  1.10s/it]

https://blog.naver.com/sweetymarch/222347856011 링크하고 있습니다


tqdm description:   6%|███▊                                                           | 12/200 [00:16<04:19,  1.38s/it]


KeyboardInterrupt: 

In [4]:
# v2
# ex) 신라면과 신라면 블랙이 있을 때 신라면 블랙먼저 실행한후 링크값을 another_links값에 넣어두고 실행
import urllib.request
import re

# 네이버 api 사용할 때 필요한 값
client_id = "84DIX0pyMLLkcG2uuHia"
client_secret = "CRf5UovTxF"

query_txt = input('검색창 키워드 입력해주세요: ') # 검색할 값을 따옴표 안에 넣어서 검색해주세요 (ex: "신라면 블랙") ( 따옴표 안에 넣고 검색시 이 값이 무조건 존재한 블로그들만 나타내는 걸로 알고 있어요!?)
encText = urllib.parse.quote(query_txt) # 네이버 검색창 검색 키워드 

blog_links = []
another_links = [] # 이전에 실행시켰던 링크값들 넣어주세요
now_links = ""

for start in range(1, 500, 100): # 블로그 500개 기준으로 작성했습니다
    stringStart = str(start) # string형태여야 합니다
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&start="+stringStart+"&display=100" # 네이버 검색 api, start: 시작, display: 보여줄 갯수(100이 최대)
    
    # start(블로그 몇 번째인지?, max값 1000)와 display(한번 검색에 몇 개의 결과를 띄울건지, max값 100)조절해주기
    # start는 1, 101, 201 이렇게 시작해주세요
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        #print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

    body = response_body.decode('utf-8')
    body = body.replace('"','')

    #블로그 링크들 추출
    links = re.findall('link: (.*?),\ndescription',body)
    print('추출 링크 수: ',len(links),'개')

    for link in links:
        link_a = link.replace('\\','') # 링크 안의 '\\' 제거
        link_b = link_a.replace('?Redirect=Log&logNo=','/') # 링크안에 ?Redirect=Log&logNo= 제거
        if not link_b in another_links: # 전에 검색했던 기준으로 중복된 링크 제거 (ex: 신라면 검색했을 때 신라면 블랙에서 검색된 링크를 제거하기 위함)
            blog_links.append(link_b) # 현재 링크를 배열에 추가 밑에 크롤링에서 사용
            now_links += link_b + "," # 현재 링크를 , 기준으로 분리 (이번 검색에서는 어떤 링크들을 사용했는지 텍스트 파일에 저장하기 위함)

print('총 링크 갯수:',len(blog_links),'개')

#본문 크롤링
path = "chromedriver.exe" # 크롬 버전에 맞는 크롬 드라이버를 실행하는 파일 위치랑 똑같은 곳에 놔둬주는게 편해요
driver = webdriver.Chrome(path)

# 블로그 링크를 하나씩 불러와서 크롤링
contents = ''
for i in tqdm(blog_links, desc="tqdm description"): 
    driver.get(i) #불러온 링크에 접속
    driver.implicitly_wait(2) # 엔진 자체에서 로딩(파싱?)되는 시간을 기다려주는 것...?
    # time.sleep(1) # driver.implicitly_wait만 써도 괜찮은지 모르겠어서 time.sleep도 넣어놓은 거예요..
    
    print(i+' 링크하고 있습니다')
    try: # 네이버 블로그
        driver.switch_to.frame("mainFrame")  # 1)iframe접근 
        blog_contents = driver.find_element_by_css_selector('div.se-main-container').text
        contents += blog_contents + '\n==========================================================================================\n'
    except NoSuchElementException:
        blog_contents = driver.find_element_by_css_selector('div#content-area').text
        contents += blog_contents + '\n==========================================================================================\n'
    except NoSuchElementException: 
        blog_contents = driver.find_element_by_css_selector('div.contents_style').text # 티스토리인 걸로 알고 있어요 (제가 한건데 기억이 안나요 ㅈㅅㅈㅅ;;;)
        contents += blog_contents + '\n==========================================================================================\n'
    except: # 그 외
        pass

# 링크 내용을 txt 파일로 저장 
text_content = open(query_txt+".txt", 'w', encoding='utf-8') # 파일이름 + 확장자명 : txt
text_content.write(contents)  
text_content.close()

# 링크 txt파일로 저장
text_link = open(query_txt+"링크.txt", 'w', encoding='utf-8')
text_link.write(now_links)  
text_link.close()

검색창 키워드 입력해주세요: "신라면 블랙"
추출 링크 수:  100 개
추출 링크 수:  100 개
총 링크 갯수: 200 개


tqdm description:   0%|                                                                        | 0/200 [00:00<?, ?it/s]

https://blog.naver.com/sadramyun/220701691920 링크 하고있습니다


tqdm description:   0%|▎                                                               | 1/200 [00:03<12:08,  3.66s/it]

https://blog.naver.com/sudal0001/221167125911 링크 하고있습니다


...

https://blog.naver.com/toithaca/221321757033 링크 하고있습니다


tqdm description:   2%|▉                                                               | 3/200 [00:09<10:00,  3.05s/it]

https://blog.naver.com/gringocerote/222082462693 링크 하고있습니다


tqdm description:   2%|█▎                                                              | 4/200 [00:34<28:16,  8.66s/it]


KeyboardInterrupt: 

In [ ]:
# ex) 신라면과 신라면 블랙이 있을 때 신라면 블랙먼저 실행한후 링크값을 another_links값에 넣어두고 실행
import os
import sys
import urllib.request
import re

# 네이버 api 사용할 때 필요한 값
client_id = "84DIX0pyMLLkcG2uuHia"
client_secret = "CRf5UovTxF"

query_txt = input('검색창 키워드 입력해주세요: ') # 검색할 값을 따옴표 안에 넣어서 검색해주세요 ( 따옴표 안에 넣고 검색시 이 값이 무조건 존재한 블로그들만 나타내는 걸로 알고 있어요!?)
encText = urllib.parse.quote(query_txt) # 네이버 검색창 검색 키워드 

blog_links = []
another_links = [] # 이전에 실행시켰던 링크값들 넣어주세요
now_links = ""

for start in range(1, 500, 100):
    stringStart = str(start) # string형태여야 합니다
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&start="+stringStart+"&display=100" # 네이버 검색 api, start: 시작, display: 보여줄 갯수(100이 최대)
    
    # start(블로그 몇 번째인지?, max값 1000)와 display(한번 검색에 몇 개의 결과를 띄울건지, max값 100)조절해주기
    # start는 1, 101, 201 이렇게 시작해주세요
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        #print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

    body = response_body.decode('utf-8')
    body = body.replace('"','')

    #블로그 링크들 추출
    links = re.findall('link: (.*?),\ndescription',body)
    print('총 링크 수: ',len(links),'개')

    for link in links:
        link_a = link.replace('\\','') # 링크 안의 '\\' 제거
        link_b = link_a.replace('?Redirect=Log&logNo=','/') # 링크안에 ?Redirect=Log&logNo= 제거
        if not link_b in another_links: # 전에 검색했던 기준으로 중복된 링크 제거 (ex: 신라면 검색했을 때 신라면 블랙에서 검색된 링크를 제거하기 위함)
            blog_links.append(link_b) # 현재 링크를 배열에 추가 밑에 크롤링에서 사용
            now_links += link_b + "," # 현재 링크를 , 기준으로 분리 (이번 검색에서는 어떤 링크들을 사용했는지 텍스트 파일에 저장하기 위함)

print('블로그 링크\n',blog_links)
print('생성된 링크 갯수:',len(blog_links),'개')

#본문 크롤링
path = "chromedriver.exe" # 크롬 버전에 맞는 크롬 드라이버를 실행하는 파일 위치랑 똑같은 곳에 놔둬주는게 편해요
driver = webdriver.Chrome(path)

# 블로그 링크를 하나씩 불러와서 크롤링
contents = ''
for i in blog_links: 
    driver.get(i) #불러온 링크에 접속
    driver.implicitly_wait(2) # 엔진 자체에서 로딩(파싱?)되는 시간을 기다려주는 것...?
    time.sleep(1) # 하다가 웹 켜지는 속도가 의외로 느려지면 조절 해주세요...ㅜㅜ
    
    print(i+'링크 하고있습니다')
    try: # 네이버 블로그
        driver.switch_to.frame("mainFrame")  # 1)iframe접근 
        blog_contents = driver.find_element_by_css_selector('div.se-main-container').text
        contents += blog_contents + '\n==========================================================================================\n'
    except NoSuchElementException:
        blog_contents = driver.find_element_by_css_selector('div#content-area').text
        contents += blog_contents + '\n==========================================================================================\n'
    except NoSuchElementException:
        blog_contents = driver.find_element_by_css_selector('div.contents_style').text
        contents += blog_contents + '\n==========================================================================================\n'
    except: # 그 외
        pass
#         blog_contents = driver.find_elements_by_css_selector('.wrap_detail_content p')
#         for blog_content in blog_contents:
#             contents += blog_content.text
#         contents += ','

# 링크 내용을 txt 파일로 저장 
text_content = open(query_txt+".txt", 'w', encoding='utf-8') # 파일 이름 중복되지 않게 조심해주세요!
text_content.write(contents)  
text_content.close()

# 링크 txt파일로 저장
text_link = open(query_txt+"링크.txt", 'w', encoding='utf-8') # 파일 이름 중복되지 않게 조심해주세요!
text_link.write(now_links)  
text_link.close()